# Import Packages and Read Data

This assumes there is a subdirectory in your current working directory called data that contains the Excel files.

In [7]:
import pandas as pd
from collections import defaultdict


deferred_factors = pd.read_excel('data/deferred_factors.xlsx',
                                 usecols=['Age', 'Combined_Factor'],
                                 index_col='Age',
                                 )
salary_increase_rate = pd.read_excel('data/salary_increase_rate.xlsx')
inforce_data = pd.read_excel('data/inforce_2022_p31.xlsx')

deferred_factors.columns = [col_name.lower() for col_name in deferred_factors.columns]
salary_increase_rate.columns = [col_name.lower() for col_name in salary_increase_rate.columns]
inforce_data.columns = [col_name.lower() for col_name in inforce_data.columns]


## Define Global Variables

### Increase Rates
`INCREASE_RATES` is an array of salary increase rates, where the index of the array is the years of service minus 1.

### Deferred Factors
`DEFERRED_FACTORS` is a dictionary of combined factors, mapping `age` -> `combined_factor`. For any age not contained in the dictionary, `combined_factor` is defaulted to 1.

### Accrued Benefit Factor
An assumption. Set to `0.0182`.

### PVAB Factor
An assumption. Set to `12.5`

In [8]:
INCREASE_RATES = salary_increase_rate['increase'].values
DEFERRED_FACTORS = defaultdict(lambda: 1, pd.Series(deferred_factors['combined_factor']).to_dict())
ACCRUED_BENEFIT_FACTOR = 0.0182
PVAB_FACTOR = 12.5

## Calculate FAS

The below code block defines a function call `calculate_fas` which calculates the Five Year Average Salary of a state pension contributor. This will be used to calculate their Accrued Benefit.

In [9]:

def calculate_fas(yos: int, comp: float, years_ahead=0) -> float:
    """
    Calculates the Five Year Average Salary (FAS) for a pension contributor.
    If the contributor has < 5 years of experience, returns 0
    If the contributor has more than 9 years of experience, then calculates
    the 3-year average salary.

    :param yos: Years of Service with state
    :param comp: Current compensation
    :param years_ahead: How many years in the future to perform the FAS calculation
    :returns: FAS in $
    """
    for i in range(years_ahead):
        comp = comp * (1 + INCREASE_RATES[yos])
        yos += 1
    cum_sum = count = 0
    yos -= 1
    if yos < 4 or yos > 49:
        return 0
    elif yos < 10:
        while yos > 0 and count < 5:
            comp = comp / (1 + INCREASE_RATES[yos - 1])
            cum_sum += comp
            count += 1
            yos -= 1
    else:
        while count < 3:
            comp = comp / (1 + INCREASE_RATES[yos - 1])
            cum_sum += comp
            count += 1
            yos -= 1
    return cum_sum / count


### Example FAS Calculations

The function `calculate_fas` contains a parameter `years_ahead` which allows you to calculate a contributor's `FAS` years in advance. 

In the below example, we first calculate the `FAS` for a contributer with 19 years of experience (`yos`) and a current compensation of $100,000 (`comp`). The resulting value is stored in the `fas_example` variable.

Next, we calculate the contributor's prior year compensation (`prior_year_comp`) by leveraging the `INCREASE_RATES` array. We then calculate the `FAS` for this contributor at `yos = 18` and `comp = prior_year_comp`; however, we set `years_ahead = 1`. This should result in the same number obtained in `fas_example`.

In [10]:
yos = 19
comp = 100000
fas_example = round(calculate_fas(yos, comp), 2)
prior_year_comp = comp / (1 + INCREASE_RATES[yos - 1])
fas_example_next_year = round(calculate_fas(yos - 1, prior_year_comp, 1), 2)
assert fas_example == fas_example_next_year, f"{'${:,.2f}'.format(fas_example)} != {'${:,.2f}'.format(fas_example_next_year)}"
print(f"{'${:,.2f}'.format(fas_example)} == {'${:,.2f}'.format(fas_example_next_year)}")

$93,387.90 == $93,387.90


### Add FAS calculations DataFrame

Now we can add `fas` and `fas_next_year` columns to our dataframe.

In [11]:
inforce_data['fas'] = inforce_data.apply(lambda row: calculate_fas(int(row['yos']), row['comp']), axis=1)
inforce_data['fas_next_year'] = inforce_data.apply(lambda row: calculate_fas(int(row['yos']), row['comp'], years_ahead=1), axis=1)
inforce_data.head()

,age,count,comp,yos,fas,fas_next_year
0,65,368,79921,35,75355.149365,77615.803846
1,57,310,75555,35,71238.576973,73375.734282
2,52,751,74812,32,70538.024227,72654.164954
3,52,4437,74659,27,70393.765048,72505.578000
4,62,530,73281,35,69094.489566,71167.324253


## Accrued Benefit & PVAB calculations

We can now leverage the `fas` calculations to calculate the Accrued Benefit and PVAB.

In [12]:
inforce_data['accrued_benefit'] = inforce_data['yos'] * inforce_data['fas'] * ACCRUED_BENEFIT_FACTOR
inforce_data['pvab'] = inforce_data['accrued_benefit'] * inforce_data['age'].map(DEFERRED_FACTORS) * PVAB_FACTOR
inforce_data['accrued_benefit_next_year'] = (inforce_data['yos'] + 1) * inforce_data['fas'] * ACCRUED_BENEFIT_FACTOR
inforce_data['pvab_next_year'] = inforce_data['accrued_benefit_next_year'] * (inforce_data['age'] + 1).map(DEFERRED_FACTORS) * PVAB_FACTOR
inforce_data['pvab_increase'] = inforce_data['pvab_next_year'] - inforce_data['pvab']
inforce_data.head()

,age,count,comp,yos,fas,fas_next_year,accrued_benefit,pvab,accrued_benefit_next_year,pvab_next_year,pvab_increase
0,65,368,79921,35,75355.149365,77615.803846,48001.230145,600015.376818,49372.693864,617158.673298,17143.296481
1,57,310,75555,35,71238.576973,73375.734282,45378.973532,318218.422429,46675.515633,351451.128367,33232.705937
2,52,751,74812,32,70538.024227,72654.164954,41081.345310,202554.586355,42365.137351,224041.786251,21487.199895
3,52,4437,74659,27,70393.765048,72505.578000,34591.496145,170555.909017,35872.662669,189707.290579,19151.381561
4,62,530,73281,35,69094.489566,71167.324253,44013.189854,441780.545855,45270.709564,488691.890850,46911.344996
